# Descubra quem fez o ENEM 2016 apenas para treino

Neste desafio deverá descobrir quais estudantes estão fazendo a prova apenas para treino.

## Tópicos

Neste desafio você aprenderá:

- Python
- Pandas
- Sklearn
- Regression
- Classification

## Requisitos

Você precisará de python 3.6 (ou superior) e do gerenciador de pacotes pip.

Para instalar os requisitos, execute o comando como no exemplo abaixo:

    pip install -r requirements.txt

## Detalhes

O contexto do desafio gira em torno dos resultados do ENEM 2016 (disponíveis no arquivo train.csv). Este arquivo, e apenas ele, deve ser utilizado para todos os desafios. Qualquer dúvida a respeito das colunas, consulte o [Dicionário dos Microdados do Enem 2016](https://s3-us-west-1.amazonaws.com/acceleration-assets-highway/data-science/dicionario-de-dados.zip).

Alguns estudantes decidem realizar prova do ENEM de forma precoce, como um teste (coluna IN_TREINEIRO). Neste desafio, você deve criar um modelo de classificação binária para inferir a mesma. Os resultados possíveis da sua resposta devem ser “0” ou “1”.

Salve sua resposta em um arquivo chamado answer.csv com duas colunas: `NU_INSCRICAO` e `IN_TREINEIRO`.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
df_train = pd.read_csv('train.csv')
df_train.head(10)

In [ ]:
df_test = pd.read_csv('test.csv')
df_test.head(10)

In [ ]:
# Queremos classificar a variável IN_TREINEIRO, avaliando os dados da coluna
df_train.IN_TREINEIRO.value_counts()
# Com isso podemos ver que os dados são altamente desbalanceados

In [ ]:
# A coluna IN_TREINEIRO ainda não existe no dataser de testes, adicionando-a
df_test['IN_TREINEIRO'] = -1

In [ ]:
# Mantendo no dataset de treino somente as colunas também presentes no dataset de testes
df_train = df_train[df_test.columns]
colunas = list(df_test.columns)

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
# AS colunas TP_ENSINO, TP_DEPENDENCIA_ADM_ESC e Q027 possuem muitos valores faltantes e serão removidas
colunas.remove('TP_ENSINO')
colunas.remove('TP_DEPENDENCIA_ADM_ESC')
colunas.remove('Q027')

In [ ]:
# As colunas TP_PRESENCA_CN, TP_PRESENCA_CH, TP_PRESENCA_LC, TP_PRESENCA_MT podem ser deduzidas pelas respectivas notas das provas
colunas.remove('TP_PRESENCA_CN')
colunas.remove('TP_PRESENCA_CH')
colunas.remove('TP_PRESENCA_LC')
colunas.remove('TP_PRESENCA_MT')

In [ ]:
# A coluna CO_UF_RESIDENCIA contém a mesma informação de SG_UF_RESIDENCIA e será removida
colunas.remove('CO_UF_RESIDENCIA')

In [ ]:
# Criando o dataset que conterá a resposta, e removendo a coluna NU_INSCRICAO dos dados de treino, uma vez que é somente o ID de cada participante
colunas.remove('NU_INSCRICAO')
answer = df_test[['NU_INSCRICAO']]

In [ ]:
## Preenchendo alguns dados faltantes
# Notas faltantes serão 0, assumindo que os alunos não compareceram
for coluna in colunas:
    if coluna.startswith('NU_NOTA'):
        df_train[coluna].fillna(0, inplace=True)
        df_test[coluna].fillna(0, inplace=True)
df_train['TP_STATUS_REDACAO'].fillna(4, inplace=True)
df_test['TP_STATUS_REDACAO'].fillna(4, inplace=True)

In [ ]:
# As Colunas que iniciam com TP_ são categóricas, ajustando o tipo de dados
for coluna in colunas:
    if coluna.startswith('TP_'):
        df_train[coluna] = df_train[coluna].astype('object')
        df_test[coluna] = df_test[coluna].astype('object')

In [ ]:
df_dummies = pd.get_dummies(pd.concat([df_train[colunas], df_test[colunas]]))

In [ ]:
# Aplicando One Hot Encoding
df_train = df_dummies.iloc[:df_train.shape[0]]
df_test = df_dummies.iloc[df_train.shape[0]:]
del df_dummies

In [ ]:
# Separando teste e treino
X_train = df_train.drop(columns=['IN_TREINEIRO'])
y_train = df_train['IN_TREINEIRO']

In [ ]:
# SMOTE
smote = SMOTE(sampling_strategy='minority')
X_smote, y_smote = smote.fit_resample(X_train, y_train)

In [ ]:
# Mantendo somente as colunas com correlação maior que 10
df_smote = X_smote
df_smote['IN_TREINEIRO'] = y_smote
corr_abs = df_smote.corr().abs()
colunas = list(corr_abs[corr_abs['IN_TREINEIRO'] > 0.30].index)

In [ ]:
X_smote_reduced = df_smote[colunas].drop(columns=['IN_TREINEIRO'])
X_test_reduced = df_test[colunas].drop(columns=['IN_TREINEIRO'])

In [ ]:
# Regressão Logistica
reg = LogisticRegression()

reg.fit(X_smote_reduced, y_smote)

y_pred = reg.predict(X_smote_reduced)

print ('mean_absolute_error:', mean_absolute_error(y_smote, y_pred))
print ('mean_squared_error:', mean_squared_error(y_smote, y_pred))
print ('r2_score:', r2_score(y_smote, y_pred))
print ('reg score:', reg.score(X_smote_reduced, y_smote))

In [ ]:
# Random Forest
reg = RandomForestClassifier()

reg.fit(X_smote_reduced, y_smote)

y_pred = reg.predict(X_smote_reduced)

print ('mean_absolute_error:', mean_absolute_error(y_smote, y_pred))
print ('mean_squared_error:', mean_squared_error(y_smote, y_pred))
print ('r2_score:', r2_score(y_smote, y_pred))
print ('reg score:', reg.score(X_smote_reduced, y_smote))

In [ ]:
answer['IN_TREINEIRO'] = reg.predict(X_test_reduced)

In [ ]:
answer

In [ ]:
answer.to_csv('answer.csv', index=False)